<a href="https://colab.research.google.com/github/alexf05/pclp3/blob/main/grile_UMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FECHET Alex-Ciprian
# alex_ciprian.fechet@stud.acs.upb.ro

In [2]:
# chat gpt
# Instalează biblioteca necesară pentru manipularea PDF-urilor
!pip install PyMuPDF

from google.colab import files
import fitz  # PyMuPDF
import re
import pandas as pd

# Pasul 1: Încarcă fișierul PDF în Colab
print("Încarcă fișierul PDF cu grilele (de ex. Grile_Examen.pdf):")
uploaded = files.upload()
file_path = next(iter(uploaded.keys()))

# Pasul 2: Definirea funcției care parsează PDF-ul și extrage întrebările și răspunsurile
def parse_exam_pdf(path):
    """
    Deschide PDF-ul de pe calea 'path', parcurge fiecare pagină și fiecare linie,
    detectează întrebările și răspunsurile (pe baza numerotării cu cifre și literelor A., B., C. etc.),
    și marchează fiecare răspuns drept corect sau incorect în funcție de stilul 'bold' din PDF.

    Returnează un DataFrame pandas cu următoarele coloane:
      - question_number: numărul întrebării (ex: "1")
      - question_text: textul complet al întrebării
      - answer_letter: litera răspunsului (ex: "A")
      - answer_text: textul complet al răspunsului
      - is_correct: True dacă răspunsul este bold (corect), False altfel
    """
    # Deschide PDF-ul
    doc = fitz.open(path)

    # Structură de date temporară pentru stocarea rezultatelor
    records = []

    # Variabile pentru starea curentă
    current_q_num = None
    current_q_text = ""
    in_question = False

    # Pattern-uri regex
    question_start_re = re.compile(r"^(\d+)\.\s*(.*)")       # ex: "1. Despre artroza se poate afirma:"
    answer_start_re   = re.compile(r"^([A-Z])\.\s*(.*)")     # ex: "A. Afecteaza preponderent..."

    # Parcurge fiecare pagină
    for page_index in range(len(doc)):
        page = doc[page_index]
        # Extrage textul în format "dict" pentru a obține span-urile cu info despre font
        page_dict = page.get_text("dict")

        # Parcurge fiecare bloc de text de pe pagină
        for block in page_dict["blocks"]:
            if block["type"] != 0:
                # Dacă nu este bloc de text simplu, sărim peste (ex: imagine)
                continue

            # Parcurge fiecare linie din bloc
            for line in block["lines"]:
                # Construim textul complet al liniei prin concatenarea span-urilor
                line_text = ""
                spans = line["spans"]
                for span in spans:
                    line_text += span["text"]

                # Înlăturăm eventuale spații de la început/sfârșit
                line_text = line_text.strip()
                if not line_text:
                    continue  # sarim rândurile goale

                # Verificăm dacă linia începe o nouă întrebare
                q_match = question_start_re.match(line_text)
                if q_match:
                    # Dacă deja procesăm o întrebare, iar acum găsim alta, salvăm ce avem până acum
                    if current_q_num is not None:
                        # Nu există "răspuns" în această ramură, doar întrebarea s-a terminat fără răspuns
                        # (nu ar trebui în mod normal să se întâmple), dar ne asigurăm că resetează starea
                        current_q_num = None
                        current_q_text = ""
                        in_question = False

                    # Preluăm numărul și începutul textului întrebării
                    current_q_num = q_match.group(1)
                    current_q_text = q_match.group(2).strip()
                    in_question = True
                    continue  # mergem la următoarea linie

                # Dacă suntem deja într-o întrebare, dar linia nu e răspuns, poate fi continuarea textului întrebării
                if in_question:
                    a_match = answer_start_re.match(line_text)
                    if a_match:
                        # Linia începe un răspuns. Extragem litera și textul răspunsului
                        ans_letter = a_match.group(1)
                        ans_text = a_match.group(2).strip()

                        # Detectăm dacă oricare span din această linie are font bold (font name conține "Bold" sau "black")
                        is_bold = False
                        for span in spans:
                            font_name = span.get("font", "")
                            if "Bold" in font_name or "Black" in font_name:
                                is_bold = True
                                break

                        # Adăugăm rândul în lista de rezultate
                        records.append({
                            "question_number": current_q_num,
                            "question_text": current_q_text,
                            "answer_letter": ans_letter,
                            "answer_text": ans_text,
                            "is_correct": is_bold
                        })
                        continue  # mergem la următoarea linie

                    else:
                        # Dacă nu începe un răspuns, iar in_question == True, înseamnă că textul întrebării continuă
                        # Adăugăm linia la textul întrebării, separată printr-un spațiu
                        current_q_text += " " + line_text
                        continue

                # Dacă ajungem aici înseamnă că nu suntem în mod activ la redactarea unei întrebări -> sărim
                # (ar putea fi titluri, secțiuni noi ș.a.)
                continue

    # Construim DataFrame-ul final
    df = pd.DataFrame.from_records(records, columns=[
        "question_number", "question_text", "answer_letter", "answer_text", "is_correct"
    ])
    return df

# Pasul 3: Apelează funcția și afișează rezultatul
df_questions = parse_exam_pdf(file_path)

# Afișăm primele 10 rânduri pentru verificare
df_questions.head(20)
df_questions.to_csv('questions.csv', index=False)


Încarcă fișierul PDF cu grilele (de ex. Grile_Examen.pdf):


Saving Grile Chir 2022-2023.pdf to Grile Chir 2022-2023 (1).pdf


,question_number,question_text,answer_letter,answer_text,is_correct
0,1,Manifestările clinice ale hemoragiei digestive...,A,,False
1,1,Manifestările clinice ale hemoragiei digestive...,B,,True
2,1,Manifestările clinice ale hemoragiei digestive...,C,,False
3,1,Manifestările clinice ale hemoragiei digestive...,D,,True
4,1,Manifestările clinice ale hemoragiei digestive...,E,,True
5,1,Manifestările clinice ale hemoragiei digestive...,F,,True
6,1,Manifestările clinice ale hemoragiei digestive...,G,,False
7,1,Manifestările clinice ale hemoragiei digestive...,H,,False
8,1,Manifestările clinice ale hemoragiei digestive...,I,,False
9,1,Manifestările clinice ale hemoragiei digestive...,J,,True
